In [1]:
import pandas as pd
from tqdm import tqdm
import os
import csv
from sys import path as sys_path
sys_path.append('/home/valerie/Projects/ProdSwisstopo/')

from utils.SwissDataset import get_tiles_ids_from_folder

In [2]:
# Get the list of all tiles ids ABOVE 200m

# get the list of all SI tiles url and all the SI tiles above 2000m
above_2000m = pd.read_csv('grid_area_above_2000m.csv')
above_2000m['id'].to_csv('ids_above2000m.csv',header=None,index=None)
above_2000m_ids = above_2000m['id'].to_list()

In [3]:
# Get the list of tiles ids that are ALREADY STORED in the machine 
# no need to download them

stored_tiles_ids = get_tiles_ids_from_folder (
    img_dir='/data/valerie/swisstopo/SI_2020_50cm/',
    dem_dir='/data/valerie/swisstopo/ALTI_2020_50cm/'
    ) 
print(stored_tiles_ids[:10])
# save list in a csv file:
pd.DataFrame(stored_tiles_ids).to_csv('ids_stored_tiles.csv',header=None,index=None)


Read images and DEM folder ...

Images files found: 5546 
Dem files found: 5546 
Common images - dem ids found : 5545
['2550_1132', '2550_1133', '2550_1134', '2551_1110', '2551_1111', '2551_1112', '2551_1131', '2551_1132', '2551_1133', '2551_1134']


In [4]:
# Get the list of tiles to DOWNLOAD and save it 

# tiles ids present  above 2000m but absent from stored tiles ids
ids_to_download  = [item for item in above_2000m_ids if item not in stored_tiles_ids]
print(len(above_2000m_ids),len(stored_tiles_ids), len(ids_to_download))


13480 5545 9840


In [5]:

pd.DataFrame(ids_to_download).to_csv('ids_to_download.csv',header=None,index=None)

In [6]:
def find_urls_with_tile_id(csv_file_path, tile_id):
    """
    Find URLs in a CSV file that contain a specified tile ID.

    Parameters:
    - csv_file_path (str): Path to the CSV file containing URLs.
    - tile_id (str): Tile ID to search for.

    Returns:
    - List of matching URLs.
    """
    matching_urls = []

    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\n')
        for row in reader:
            
            if tile_id in row[0]:
                matching_urls.append(row[0])
            

    if len(matching_urls)!= 1 :
        print('found more than one/zeros url for', tile_id,)
        raise KeyError

    return matching_urls[0]

# Example usage:
csv_file_path = 'url_SA_all.csv'
tile_id_to_search = '2507-1129'
matching_urls = find_urls_with_tile_id(csv_file_path, tile_id_to_search)
print(matching_urls)




https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2507-1129/swissalti3d_2019_2507-1129_0.5_2056_5728.tif


In [7]:
## from the list of SA and SI url for all tile ids to download
url_dict=[]
SA_file_path = 'url_SA_all.csv'
SI_file_path = 'url_SI_all.csv'

for k, id in tqdm (enumerate( ids_to_download )): 
    id = id.replace('_','-')
    SI_url =  find_urls_with_tile_id(SI_file_path, id)
    SA_url =  find_urls_with_tile_id(SA_file_path, id)
    url_dict += [[SI_url, SA_url]]
    if k % 100 == 0:   
        pd.DataFrame(url_dict).to_csv('url_to_download.csv',header=None, index=None)
        
pd.DataFrame(url_dict).to_csv('url_to_download.csv',header=None, index=None)

9840it [10:08, 16.16it/s]
